# Omówienie zbioru danych - EDA

Dane zawierają informacje na temat wypożyczeń rowerów

Źródło: https://www.kaggle.com/datasets/geometrein/helsinki-city-bikes

Ogólny cel projektu:
1. Zrozumienie danych i analizę różnych aspektów takich jak, charakterystyka stacji czy klientów.
2. Obróbka danych - jest ich bardzo dużo i należy je odpowiednio przygotować.
3. Model prognozy przyszłorocznej liczby wypożyczeń.
4. Prognoza wypożyczeń na kolejny dzień.
5. System do alertów (progonozowanie czy wypożyczeń będzie więcej niż zwrotów) w celu szybszej reakcji.


Cele EDA:
Dane te pozwalają na analizę wielu aspektów. 
Lista zadań do realizacji:
1. Pobranie danych i podstawowe statystyki (head, info)
2. Konwersja dat. (z formatu object na datetime)
3. Agregacja danych:
    - Przygotowanie funkcji do agregacji
    - Stworzenie i zapisanie ramki danych z sumą wypożyczeń, średnim dystansem, prędkością i pogodą dla każdego dnia.
    - Stworzenie i zapisanie ramki danych z sumą wypożyczeń, średnim dystansem, prędkością i pogodą dla każdego dnia (z dokładnością do godziny) i stacji. (dla stacji departure i osobno dla stacji return)
    - Zapisanie zagregowanych danych.
4. Mapa stacji
    - Stworzenie wykresu położenia stacji. (w zbiorze mamy współrzędne geograficzne).
5. Analiza stacji:
    - minimalne i maksymalne daty wypożyczeń dla stacji
    - liczba wypożyczeń (Najpopularniejsze stacje)
6. Analiza użytkowników:
- Jaki jest rozkład:
    - pokonywanej odległości
    - czasu
    - prędkości
- Czy zmienia się po latach?
7. Funkcja do oceny czy w pobliżu jest stacja
- Funkcja, która dla wybranej stacji, sprawdzi w jakiej odległości znajduje się najbliższa stacja
- Pozwoli dokonać ostatecznej oceny, czy stacje z małą liczbą wypożyczeń można usunąć.


In [ ]:
#pip install kagglehub -U

1. Pobranie danych i podstawowe statystyki (head, info)

In [ ]:
# Biblioteka kagglehub
import kagglehub


In [ ]:
path = kagglehub.dataset_download("geometrein/helsinki-city-bikes")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns

In [ ]:
# ladowanie parquet
#df = pd.read_parquet('data/source_file.parquet')

In [ ]:
# puść ten kod, 
# jeżeli wywołujesz plik  w folderze rozwiąznaia, 
# a ramka danych znajduje się w folderze data
import os 
os.chdir('../')

In [ ]:
# Ładowanie danych
df = pd.read_csv(path + r'\database.csv', dtype={'departure_id':'object',
                                                 'return_id': 'object'})

In [ ]:
# info
df.info()

In [ ]:
# head
df.head()

2. Konwersja dat. (z formatu object na datetime)

In [ ]:
# Konwersja dat departure
df['departure'] = pd.to_datetime(df['departure'])
df['departure_date'] = df['departure'].dt.round('D')
df['departure_date_hours'] = df['departure'].dt.round('h')

In [ ]:
# Konwersja dat returns
df['return'] = pd.to_datetime(df['return'])
df['return_date'] = df['return'].dt.round('D')
df['return_date_hours'] = df['return'].dt.round('h')

In [ ]:
# Zapis pliku do parquet
df.to_parquet('data/source_file.parquet')

3. Agregacja danych:
    - Przygotowanie funkcji do agregacji
    - Stworzenie i zapisanie ramki danych z sumą wypożyczeń, średnim dystansem, prędkością i pogodą dla każdego dnia.
    - Stworzenie i zapisanie ramki danych z sumą wypożyczeń, średnim dystansem, prędkością i pogodą dla każdego dnia i stacji. (dla stacji departure i osobno dla stacji return)
    - Zapisanie zagregowanych danych.

In [ ]:
from help_function import agg_data

In [ ]:
## ogólny szereg czasowy - liczba wypożyczeń dziennie

df_total_agg  =agg_data(df,['departure_date'],
                        {'departure_name': 'count',
                         'distance (m)': 'mean',
                         'duration (sec.)': 'mean',
                         'avg_speed (km/h)': 'mean',
                         'Air temperature (degC)': 'mean'})

In [ ]:
df_total_agg

In [ ]:
# zmiana nazw
df_total_agg = df_total_agg.rename(columns = {'departure_name': 'numbers_of_renting'})

In [ ]:
df_total_agg.to_parquet('data/total_agg.parquet')

In [ ]:
# head
df_total_agg.head()

In [ ]:
df.departure_id.unique()

In [ ]:
df.departure_id.nunique()

In [ ]:
df.departure_name.nunique()

In [ ]:
## aggregate data - departures
df_agg_dep = agg_data(df, ['departure_name', 'departure_date_hours'],
                      {'departure_id': 'count',
                         'distance (m)': 'mean',
                         'duration (sec.)': 'mean',
                         'avg_speed (km/h)': 'mean',
                         'Air temperature (degC)': 'mean'})

In [ ]:
# zmiana nazw
df_agg_dep = df_agg_dep.rename(columns = {'departure_id': 'numbers_of_departures'})

In [ ]:
# head
df_agg_dep

In [ ]:
# zapis do parquet
df_agg_dep.to_parquet('data/hourly_data_per_station.parquet')


In [ ]:
# aggregate data - returns
df_agg_ret = agg_data(df, ['return_name', 'return_date_hours'],
                      {'return_id': 'count',
                         'distance (m)': 'mean',
                         'duration (sec.)': 'mean',
                         'avg_speed (km/h)': 'mean',
                         'Air temperature (degC)': 'mean'})

In [ ]:
# zmiana nazw
df_agg_ret  = df_agg_ret.rename(columns = {'return_id': 'number_of_returns'})

In [ ]:
# head
df_agg_ret.head()

In [ ]:
# zapis do parquet
df_agg_ret.to_parquet('data/hourly_data_per_station_returns')

In [ ]:
plt.figure(figsize=[12,8])
plt.plot(df_total_agg['departure_date'],df_total_agg['numbers_of_renting'])
plt.title('Dzienna liczba wypożyczeń')
plt.show()

4. Mapa stacji
    - Stworzenie wykresu położenia stacji. (w zbiorze mamy współrzędne geograficzne).

In [ ]:
#pip install folium

In [ ]:
import folium

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_parquet('data/source_file.parquet')

In [ ]:
# przygotowanie danych do mapy
df_map = df.loc[df['departure']>='2020-01-01',['departure_name','departure_longitude', 'departure_latitude', 'departure_id']].groupby('departure_name').agg({
    'departure_latitude':'mean',
    'departure_longitude': 'mean',
    'departure_id': 'count'
})

In [ ]:
df_map.head()

In [ ]:
df_map = df_map.reset_index().rename(columns = {'departure_id':'amount'})

In [ ]:
# definicja mapy
the_map = folium.Map(location = [df_map.departure_latitude.mean(),df_map.departure_longitude.mean()], zoom_start=10)

In [ ]:
# naniesienie informacji z punktów
for i in range(len(df_map)):
    lat = df_map.loc[i,'departure_latitude']
    lon = df_map.loc[i,'departure_longitude']
    amount = df_map.loc[i,'amount']
    name = df_map.loc[i,'departure_name']
    folium.Marker(location=[lat,lon], popup=f"Nazwa: {name} \n wypozyczenia: {amount}").add_to(the_map)

In [ ]:
# print mapy
the_map

In [ ]:
# Mapa inaczej
the_map_2 = folium.Map(location=[df_map.departure_latitude.mean(),df_map.departure_longitude.mean()], zoom_start=15)

In [ ]:
# naniesienie informacji z punktów
for i in range(len(df_map)):
    lat = df_map.loc[i,'departure_latitude']
    lon = df_map.loc[i,'departure_longitude']
    amount = df_map.loc[i,'amount']
    name= df_map.loc[i,'departure_name']
    folium.CircleMarker(
        location=[lat, lon],
        radius=amount / 1000,  # Skala wielkości
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=f"Liczba wypożyczeń: {amount}\n stacja: {name}"
    ).add_to(the_map_2)


In [ ]:
# print mapy
the_map_2

In [ ]:
# zapisywanie mapy
the_map_2.save('data/map.html')

5. Analiza stacji:
    - minimalne i maksymalne daty wypożyczeń dla stacji
    - liczba wypożyczeń (Najpopularniejsze stacje)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_parquet('data/source_file.parquet')

In [ ]:
df.info()

In [ ]:
# minimalna data wypożyczenia ze stacji
min_dates = df.loc[:,['departure_name','departure_date']].groupby('departure_name').min()

In [ ]:
min_dates.describe()

In [ ]:
max_dates = df.loc[:,['departure_name','departure_date']].groupby('departure_name').max()

In [ ]:
max_dates.describe()

In [ ]:
new_stations = min_dates[min_dates['departure_date']>='2020-01-01'].reset_index()
new_stations

In [ ]:
# przygotowanie danych do mapy
df_map = df.loc[df['departure']>='2020-01-01',['departure_name','departure_longitude', 'departure_latitude', 'departure_id']].groupby('departure_name').agg({
    'departure_latitude':'mean',
    'departure_longitude': 'mean',
    'departure_id': 'count'
})

In [ ]:
df_map = df_map.reset_index().rename(columns = {'departure_id':'amount'})

In [ ]:
df_map.describe()

In [ ]:
potential_reduction = df_map[(df_map['amount']<=df_map.amount.quantile(0.25)) & ~(df_map['departure_name'].isin(new_stations['departure_name']))]
potential_reduction

6. Analiza użytkowników
- Jaki jest rozkład:
    - pokonywanej odległości
    - czasu
    - prędkości
- Czy zmienia się po latach?

In [ ]:
# Wykresy gęstości
sns.kdeplot(df,x='distance (m)')
plt.show()

In [ ]:
df.columns

In [ ]:
sns.kdeplot(df['duration (sec.)'] )
plt.show()

In [ ]:
sns.kdeplot(df['avg_speed (km/h)'])
plt.show()

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
# describe
df[['distance (m)','duration (sec.)','avg_speed (km/h)']].describe()

In [ ]:
sns.boxplot(df[['distance (m)','duration (sec.)','avg_speed (km/h)']])

In [ ]:
df['year'] = df['departure_date'].dt.year

In [ ]:
pd.set_option('display.max_columns',None)

In [ ]:
df[['distance (m)','duration (sec.)','avg_speed (km/h)','year']].groupby('year').describe()

7. Funkcja do oceny czy w pobliżu jest stacja

In [ ]:
#pip install geopy

In [ ]:
from geopy.distance import geodesic

In [ ]:
potential_reduction.head()

In [ ]:
def calculate_min_distance(data_point, points_to_check):
    distances = []
    for i in range(len(points_to_check)):
        point = points_to_check.iloc[i,:]
        distance = geodesic(data_point, point).meters
        if distance>0:
            distances.append(distance)
    return min(distances)

In [ ]:
potential_reduction

In [ ]:
station_to_check = potential_reduction.loc[potential_reduction['departure_name']=='Asentajanpuisto',['departure_latitude','departure_longitude']]
station_to_check

In [ ]:
potential_reduction = potential_reduction.reset_index(drop=True)

In [ ]:
calculate_min_distance(station_to_check.values, df_map[['departure_latitude','departure_longitude']])

In [ ]:
for i in range(len(potential_reduction)):
    st_to_check = potential_reduction.loc[i,['departure_latitude','departure_longitude']].values
    dist = calculate_min_distance(st_to_check,df_map[['departure_latitude','departure_longitude']])
    potential_reduction.loc[i,'min_dist'] = dist

In [ ]:
potential_reduction